In [1]:
import os

from pathlib import Path

# # Print the current working directory
# print("Current working directory:", os.getcwd())


project_root = Path('C:/Test-Summarizer')
os.chdir(project_root)
print("Current working directory:", os.getcwd())


# Specify the paths to your configuration and parameters files
config_filepath = project_root / 'config' / 'config.yaml'
params_filepath = project_root / 'params.yaml'

Current working directory: C:\Test-Summarizer


In [2]:
from dataclasses import dataclass


from textSummarizer.constants import *
from textSummarizer.logging import logger
from textSummarizer.utils.common import read_yaml, create_directories


from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch


C:\Users\harry\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[ 2024-03-27 20:08:52,983 - INFO - config -PyTorch version 2.2.1 available.]


In [3]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path,  
        params_filepath: Path):  
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([Path(self.config['artifacts_root'])])
       
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config['model_trainer']
        params = self.params['TrainingArguments']
        
        create_directories([Path(config['root_dir'])])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config['root_dir']),
            data_path=Path(config['data_path']),
            model_ckpt=Path(config['model_ckpt']),
            num_train_epochs=params['num_train_epochs'],
            warmup_steps=params['warmup_steps'],
            per_device_train_batch_size=params['per_device_train_batch_size'],
            weight_decay=params['weight_decay'],
            logging_steps=params['logging_steps'],
            evaluation_strategy=params['evaluation_strategy'],
            eval_steps=params['eval_steps'],
            save_steps=params['save_steps'],
            gradient_accumulation_steps=params['gradient_accumulation_steps']
        )

        return model_trainer_config
        
        
        

In [5]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
        self.corrected_model_ckpt = str(config.model_ckpt).replace("\\", "/")

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.corrected_model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.corrected_model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        # Loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Using TrainingArguments from the configuration directly
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, 
            num_train_epochs=self.config.num_train_epochs, 
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, 
            per_device_eval_batch_size=1,  
            weight_decay=self.config.weight_decay, 
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, 
            eval_steps=self.config.eval_steps, 
            save_steps=int(float(self.config.save_steps)),
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        ) 

        trainer = Trainer(
            model=model_pegasus, args=trainer_args,
            tokenizer=tokenizer, data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"], 
            eval_dataset=dataset_samsum_pt["validation"]
        )
        
        trainer.train()

        # Save model and tokenizer
        model_save_path = os.path.join(self.config.root_dir, "pegasus-samsum-model")
        tokenizer_save_path = os.path.join(self.config.root_dir, "tokenizer")
        model_pegasus.save_pretrained(model_save_path)
        tokenizer.save_pretrained(tokenizer_save_path)

In [6]:
try:
    config_manager = ConfigurationManager(config_filepath=config_filepath, params_filepath=params_filepath)
    model_trainer_config = config_manager.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[ 2024-03-27 20:08:53,707 - INFO - common -yaml file: C:\Test-Summarizer\config\config.yaml loaded successfully]
[ 2024-03-27 20:08:53,711 - INFO - common -yaml file: C:\Test-Summarizer\params.yaml loaded successfully]
[ 2024-03-27 20:08:53,721 - INFO - common -Created directory at: artifacts]
[ 2024-03-27 20:08:53,723 - INFO - common -Created directory at: artifacts\model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Test-Summarizer\est\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  0%|          | 0/920 [00:00<?, ?it/s]